In [21]:
import urllib2
import lxml
from lxml import html
import cssselect
import requests
import folium
import json
import urllib
import pandas
import csv
import geopandas
import shapely
import shapely.geometry
import fiona
import fiona.crs
import shapely.wkt

In [22]:
url = "https://en.wikipedia.org/wiki/List_of_cities_in_China_by_population_and_built-up_area"
req = urllib2.Request(url, headers={'User-Agent' : "Magic Browser"}) 
con = urllib2.urlopen( req )
doc_text = con.read()
doc = lxml.html.fromstring(doc_text)
doc.make_links_absolute(url)

In [23]:
for row in doc.cssselect("table.wikitable tr")[1:]:
    print row.text # Nothing show up but I am trying to get some space between each city,
    # easy to read~
    pop_area = row.cssselect("td")
    city_name = row.cssselect("a")
    #print "Cities:" + str(city_name[0].text)
    #print "Metro pop: "+str(pop_area[2].text)
    #print "Urban area: "+str(pop_area[3].text)
    
    list_pop = []
    list_urban = []
    list_cities = []
    
    list_pop.append(str(pop_area[2].text))
    list_urban.append(str(pop_area[3].text))
    list_cities.append(str(city_name[0].text))
    print "Cities :" + str(list_cities)
    print "Metropolitan Population" + str((list_pop))
    print "Urban Population" + str(list_urban)
    zip_list = zip(list_cities, list_pop, list_urban)
    #for item in zip_list:
        #data = [item]
        #geopandas_df = geopandas.GeoDataFrame(data,columns=['Cities','Lat','Long'])
        #print geopandas_df
        #geopandas_df.plot()



Cities :['Guangzhou']
Metropolitan Population['44,294,245']
Urban Population['20,800,654']


Cities :['Shanghai']
Metropolitan Population['35,965,403']
Urban Population['24,500,000']


Cities :['Chongqing']
Metropolitan Population['30,165,500']
Urban Population['18,384,000']


Cities :['Beijing']
Metropolitan Population['21,500,000']
Urban Population['21,500,000']


Cities :['Hangzhou']
Metropolitan Population['21,102,000']
Urban Population['9,018,000']


Cities :['Wuhan']
Metropolitan Population['19,780,644']
Urban Population['10,670,000']


Cities :['Chengdu']
Metropolitan Population['18,000,000']
Urban Population['17,677,122']


Cities :['Tianjin']
Metropolitan Population['16,000,000']
Urban Population['15,500,000']


Cities :["Xi'an"]
Metropolitan Population['13,569,700']
Urban Population['8,627,500']


Cities :['Jinan']
Metropolitan Population['13,000,000']
Urban Population['7,067,900']


Cities :['Shenzhen']
Metropolitan Population['None']
Urban Population['12,357,938']


Citie

UnicodeEncodeError: 'ascii' codec can't encode character u'\xdc' in position 0: ordinal not in range(128)

In [14]:
list_cities = ["Guangzhou", "Shanghai", "Chongqing","Beijing","Hangzhou","Wuhan", "Chengdu",
              "Tianjin", "Xian", "Jinan", "Shenzhen", "Nanjing", "Shenyang", "Harbin", "Shantou",
               "Suzhou"
              ] # Find all cities that with more than ten million population
for cities in list_cities:
    my_url = "http://nominatim.openstreetmap.org/search.php?q=" + urllib.quote(cities) + "&polygon=1&format=json"
    print my_url
    response = urllib2.urlopen(my_url)
    my_json = response.read()
    json_as_dictionary = json.loads(my_json)
print json_as_dictionary

http://nominatim.openstreetmap.org/search.php?q=Guangzhou&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Shanghai&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Chongqing&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Beijing&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Hangzhou&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Wuhan&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Chengdu&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Tianjin&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Xian&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Jinan&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Shenzhen&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Nanjing&polygon=1&format=json
http://nominatim.openstreetmap.org/search.php?q=Shenyang&polygon

In [24]:
#Building data base
data = [['Guangzhou',23.13,113.25,44294245,20800654],['Shanghai',31.225,121.48,35965403,24500000],
        ['Chongqing',29.5585,106.549,30165500,18384000], ['Beijing',39.9,116.3,21500000,21500000],
        ['Hangzhou',30.2,120.2,21102000,9018000], ['Wuhan',30.5,114.2,19780644,10670000],
       ['Chengdu',30.67,104.06,18000000,17677122], ['Tianjin',39.12,117.19,16000000,15500000],
       ['Xian',34.219,109.11,13569700,8627500], ['Jinan',36.65,117.11,13000000,7067900], 
       ['Shenzhen',22.544,114.054,14668426,12357938], ['Nanjing',32.08911,118.7917,12652000,8216000],
       ['Shenyang',41.8045293,123.4277,12263612,8255921], ['Harbin',45.765,126.616,12000000,12000000]
       ,['Shantou',23.365,116.698,11630447,5346708], ['Suzhou',31.298,120.622,10578700,10349090]]

geopandas_df = pandas.DataFrame(data,columns=['Cities','Lat','Long','Metro_Population',
                                                   'Urban_area_Pop'])
print "Original dataframe:"
print type(geopandas_df)
print geopandas_df


#Constructing geometry

CoordinateTuples_list = zip(geopandas_df['Long'], geopandas_df['Lat'])

geometry_list = [shapely.geometry.Point(CoordinateTuple) for CoordinateTuple in CoordinateTuples_list]

geometry_gs = geopandas.GeoSeries(geometry_list)

geopandas_gdf = geopandas.GeoDataFrame(
    geopandas_df,
    geometry=geometry_gs,
)
geopandas_gdf.crs=fiona.crs.from_epsg(4326)
print "GeoDataFrame:"
print type(geopandas_gdf)
print geopandas_gdf



# Making Folium Map
map_points = folium.Map(location=[35.8617, 104.1954],
                            zoom_start=4,
                            tiles='Mapbox Bright')


points = folium.features.GeoJson(geopandas_gdf.to_json())

map_points.add_children(points)

map_points

Original dataframe:
<class 'pandas.core.frame.DataFrame'>
       Cities        Lat      Long  Metro_Population  Urban_area_Pop
0   Guangzhou  23.130000  113.2500          44294245        20800654
1    Shanghai  31.225000  121.4800          35965403        24500000
2   Chongqing  29.558500  106.5490          30165500        18384000
3     Beijing  39.900000  116.3000          21500000        21500000
4    Hangzhou  30.200000  120.2000          21102000         9018000
5       Wuhan  30.500000  114.2000          19780644        10670000
6     Chengdu  30.670000  104.0600          18000000        17677122
7     Tianjin  39.120000  117.1900          16000000        15500000
8        Xian  34.219000  109.1100          13569700         8627500
9       Jinan  36.650000  117.1100          13000000         7067900
10   Shenzhen  22.544000  114.0540          14668426        12357938
11    Nanjing  32.089110  118.7917          12652000         8216000
12   Shenyang  41.804529  123.4277          1

# The map above doesn't have popup, thus I will make another map with popup.

In [25]:
geopandas_gdf.to_csv("CitiesinChina.csv") # Turning my Geodataframe into CSV file
new_gdf = pandas.read_csv("CitiesinChina.csv", index_col =0)
print "type: "+str(type(new_gdf))
new_gdf

type: <class 'pandas.core.frame.DataFrame'>


,Cities,Lat,Long,Metro_Population,Urban_area_Pop,geometry
0,Guangzhou,23.130000,113.2500,44294245,20800654,POINT (113.25 23.13)
1,Shanghai,31.225000,121.4800,35965403,24500000,POINT (121.48 31.225)
2,Chongqing,29.558500,106.5490,30165500,18384000,POINT (106.549 29.5585)
3,Beijing,39.900000,116.3000,21500000,21500000,POINT (116.3 39.9)
4,Hangzhou,30.200000,120.2000,21102000,9018000,POINT (120.2 30.2)
5,Wuhan,30.500000,114.2000,19780644,10670000,POINT (114.2 30.5)
6,Chengdu,30.670000,104.0600,18000000,17677122,POINT (104.06 30.67)
7,Tianjin,39.120000,117.1900,16000000,15500000,POINT (117.19 39.12)
8,Xian,34.219000,109.1100,13569700,8627500,POINT (109.11 34.219)
9,Jinan,36.650000,117.1100,13000000,7067900,POINT (117.11 36.65)


In [7]:
newGeometryGeoSeries = geopandas.GeoSeries(
    [shapely.wkt.loads(pointtext) for pointtext in new_gdf['geometry']]
)

newGeometryGeoSeries # Reconstruct geometry for my dataframe

0            POINT (113.25 23.13)
1           POINT (121.48 31.225)
2         POINT (106.549 29.5585)
3              POINT (116.3 39.9)
4              POINT (120.2 30.2)
5              POINT (114.2 30.5)
6            POINT (104.06 30.67)
7            POINT (117.19 39.12)
8           POINT (109.11 34.219)
9            POINT (117.11 36.65)
10         POINT (114.054 22.544)
11      POINT (118.7917 32.08911)
12    POINT (123.4277 41.8045293)
13         POINT (126.616 45.765)
14         POINT (116.698 23.365)
15         POINT (120.622 31.298)
dtype: object

In [8]:
new_gdf_2 = geopandas.GeoDataFrame(
    new_gdf.copy(),
    geometry = newGeometryGeoSeries )
print type(new_gdf_2)
new_gdf_2 # Turn dataframe into a new Geodataframe

<class 'geopandas.geodataframe.GeoDataFrame'>


,Cities,Lat,Long,Metro_Population,Urban_area_Pop,geometry
0,Guangzhou,23.130000,113.2500,44294245,20800654,POINT (113.25 23.13)
1,Shanghai,31.225000,121.4800,35965403,24500000,POINT (121.48 31.225)
2,Chongqing,29.558500,106.5490,30165500,18384000,POINT (106.549 29.5585)
3,Beijing,39.900000,116.3000,21500000,21500000,POINT (116.3 39.9)
4,Hangzhou,30.200000,120.2000,21102000,9018000,POINT (120.2 30.2)
5,Wuhan,30.500000,114.2000,19780644,10670000,POINT (114.2 30.5)
6,Chengdu,30.670000,104.0600,18000000,17677122,POINT (104.06 30.67)
7,Tianjin,39.120000,117.1900,16000000,15500000,POINT (117.19 39.12)
8,Xian,34.219000,109.1100,13569700,8627500,POINT (109.11 34.219)
9,Jinan,36.650000,117.1100,13000000,7067900,POINT (117.11 36.65)


In [26]:
# Making map from CSV file
cities = pandas.read_csv('CitiesinChina.csv')
Coordinates_China = (35.8617, 104.1954)

map_1 = folium.Map(location = Coordinates_China, zoom_start = 4)


for row in cities.itertuples():
    
# I didn't use HTML popup becasue I tried couple different ways to loop through the HTML but none
# of them work. Thus, I will just use simplest marker.
    
    #html="""
    #<h3 style = "color:#841B2D" > Cities with Ten million population in China</h3>
    #<body>
        #<table border = "1px">
            #<thead>
                #<tr>
                    #<th><h4> City Name</h4> </th>
                    #<th><h4> Metro Population </h4> </th>
                    #<th><h4> Urban Population </h4></th>
            #</thead>
            
            #<tbody> 
                #% for row in cities:
                    #<tr>
                            #<td> ${row[2]} </td>
                            #<td> ${row[5]} </td>
                            #<td> ${row[6]} </td>
                    #</tr>
                    #% endfor
            #</tbody>
    #</body>
    #"""
    #iframe = folium.element.IFrame(html=html, width=300, height=100)
    #popup_1 = folium.Popup(iframe, max_width=1000)
    #folium.Marker([float(row[3]),float(row[4])], popup=popup_1).add_to(map_1)
    #folium.Marker([30,-100], popup=popup).add_to(m)

    map_1.simple_marker(
            location = [float(row[3]),float(row[4])],
            popup = "City:" + str(row[2]) + "\n" +
            "Metro_pop:"+ str(row[5]) + "\n" +
            "Urban_pop:" + str(row[6]))

import os
map_1.save('CitisinChina.html')
map_1.save('CIinChina.shp')
map_1

/Users/zhengweisun/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:44: FutureWarning: simple_marker is deprecated. Use add_children(Marker) instead


In [ ]:
#URL for my map in student server: http://students.washington.edu/sunz4/CC.html

# This whole project took me at least 10 hours to finish because I spent the most of the 
# time making Geodataframe and trying to add popup on the map.  